In [1]:
from root_pandas import read_root
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, confusion_matrix, precision_score, recall_score
import gc ; gc.enable()
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm
import lightgbm as lgb
from fnmatch import filter
import warnings
warnings.filterwarnings('ignore')
import gc ; gc.enable()
from TOF_COM_calculation import TOF_COM_calculator

Welcome to JupyROOT 6.16/00


/home/colmsam/miniconda/envs/cern_env/lib/python3.7/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.4 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


# Calculate COM variables for the TwoBody candidates df

In [2]:
ids = [
'runNumber'
,'eventNumber'
,'nCandidate']

COM_TB_stuff = ["TwoBody_M",
"TwoBody_PE",
"TwoBody_PX",
"TwoBody_PY",
"TwoBody_PZ",
"TwoBody_ENDVERTEX_X",
"TwoBody_ENDVERTEX_Y",
"TwoBody_ENDVERTEX_Z",
"TwoBody_OWNPV_X",
"TwoBody_OWNPV_Y",
"TwoBody_OWNPV_Z",
"Track1_PX",
"Track1_PY",
"Track1_PZ",
"Track2_PX",
"Track2_PY",
"Track2_PZ",
'Track1_ProbNNe',
'Track1_ProbNNk',
'Track1_ProbNNp',
'Track1_ProbNNpi',
'Track1_ProbNNmu',
'Track1_ProbNNghost',
'Track2_ProbNNe',
'Track2_ProbNNk',
'Track2_ProbNNp',
'Track2_ProbNNpi',
'Track2_ProbNNmu',
'Track2_ProbNNghost']

features = [
'TwoBody_DIRA_OWNPV'
,'TwoBody_DOCAMAX'
,'TwoBody_ENDVERTEX_CHI2'
,'noexpand:log(TwoBody_FDCHI2_OWNPV)'    
,'TwoBody_FD_OWNPV'
,'TwoBody_M'
,'TwoBody_IPCHI2_OWNPV'
,'TwoBody_Mcorr'
,'TwoBody_PT'
,'noexpand:log(Track1_MINIPCHI2)'
,'Track1_PT'
,'noexpand:log(Track2_MINIPCHI2)'
,'Track2_PT',
'Track1_Charge',
'Track2_Charge']


labels = [
'SignalB_ID' ]

TBcols = ids+ COM_TB_stuff + features + labels

In [54]:
df = read_root('TaggingJpsiK2012_tiny_fix_fix.root', columns= TBcols )

#as eventNumber is not unique, but for a given run number, it is, so we can combine the two to get a completely unique id for each event
df.index = df.apply(lambda x:str(int(x.runNumber)) + str(int(x.eventNumber))+'-'+str(int(x.nCandidate)), axis=1 )
promising_TB_ids = pd.read_csv('second_round_twobody_candidates_tiny.csv', header=None)[0]

#cutting out extra track canidates from TwoBodyCandidates that weren't recoginised by first stage MVA
TB_df = df.loc[promising_TB_ids, :]

#get rid of other id like columns as now no need for them
TB_df = TB_df.drop(columns=['runNumber', 'eventNumber', 'nCandidate'], axis=0)

#calculate COM variables for TB_df
COM_TB_df = TOF_COM_calculator(df = TB_df, PXs=["Track1_PX", "Track2_PX"], PYs=["Track1_PY", "Track2_PY"], PZs=["Track1_PZ", "Track2_PZ"], proton_probs=['Track1_ProbNNp','Track2_ProbNNp'], kaon_probs=['Track1_ProbNNk','Track2_ProbNNk'], pion_probs=['Track1_ProbNNpi','Track2_ProbNNpi'], electron_probs=['Track1_ProbNNe','Track2_ProbNNe'], muon_probs=['Track1_ProbNNmu','Track2_ProbNNmu'], names=['track1', 'track2'] )

del df, TB_df
gc.collect()

182

# Caclulate COM variables for the good extra tracks df

In [16]:
ids = [
'runNumber'
,'eventNumber'
,'nCandidate']

COM_stuff = ["TwoBody_M",
"TwoBody_PE",
"TwoBody_PX",
"TwoBody_PY",
"TwoBody_PZ",
"TwoBody_ENDVERTEX_X",
"TwoBody_ENDVERTEX_Y",
"TwoBody_ENDVERTEX_Z",
"TwoBody_OWNPV_X",
"TwoBody_OWNPV_Y",
"TwoBody_OWNPV_Z"]

extra_COM_stuff = ['TwoBody_Extra_Px',
'TwoBody_Extra_Py',
'TwoBody_Extra_Pz',
'TwoBody_Extra_TRUEPID',
'TwoBody_Extra_FromSameB',
'TwoBody_Extra_CHARGE',
'TwoBody_Extra_NNp',
 'TwoBody_Extra_NNk',
 'TwoBody_Extra_NNpi',
 'TwoBody_Extra_NNmu',
 'TwoBody_Extra_NNe',
'TwoBody_Extra_NNg']

EXTRAcols = ids + COM_stuff + extra_COM_stuff

In [17]:
df = read_root('TaggingJpsiK2012_tiny_fix_fix.root', columns=EXTRAcols, flatten = extra_COM_stuff )

#as eventNumber is not unique, but for a given run number, it is, so we can combine the two to get a completely unique id for each event
df.index = df.apply(lambda x:str(int(x.runNumber)) + str(int(x.eventNumber))+'-'+str(int(x.nCandidate)), axis=1 )

#cutting out extra track canidates from TwoBodyCandidates that weren't recoginised by first stage MVA
extra_tracks_df = df.loc[promising_TB_ids, :]

#giving new_df a new index for each unique extra track
extra_tracks_df.index = extra_tracks_df.apply(lambda x:str(int(x.runNumber)) + str(int(x.eventNumber))+'-'+str(int(x.nCandidate))+'-'+str(int(x.__array_index)), axis=1 )

#cutting out the bad extra tracks
promising_extra_tracks_ids = pd.read_csv('THIRD_round_extra_track_candidates_tiny.csv', header=None)[0]
extra_tracks_df = extra_tracks_df.loc[promising_extra_tracks_ids, :]

#get rid of other id like columns as now no need for them
extra_tracks_df['TB_id'] = extra_tracks_df.apply(lambda x:str(int(x.runNumber)) + str(int(x.eventNumber))+'-'+str(int(x.nCandidate)), axis=1 )
extra_tracks_df = extra_tracks_df.drop(columns=['runNumber', 'eventNumber', 'nCandidate', '__array_index'], axis=0)

#calculating the COM variables for the extra tracks
extra_tracks_COM_df = TOF_COM_calculator(df = extra_tracks_df, PXs=["TwoBody_Extra_Px"], PYs=["TwoBody_Extra_Py"], PZs=["TwoBody_Extra_Pz"], proton_probs=['TwoBody_Extra_NNp'], kaon_probs=['TwoBody_Extra_NNk'], pion_probs=['TwoBody_Extra_NNpi'], electron_probs=['TwoBody_Extra_NNe'], muon_probs=['TwoBody_Extra_NNmu'], names=['extra_track'] )

#deleting data objects which we no longer need to save RAM space
del df, extra_tracks_df, promising_TB_ids, promising_extra_tracks_ids
gc.collect()

168

# Add extra track COM info to the TB_df

In [18]:
#I need to add a TB_index column to extra tracks df so I can merge them later
#extra_tracks_COM_df['TB_id'] = extra_tracks_COM_df.apply(lambda x: x.name[:-2], axis=1 )

count = 0
#defing the extra tracks that need to be added, and looping over until theres none left
need_adding = extra_tracks_COM_df
while len(need_adding) != 0:
   
    print(len(need_adding))
    
    #defining the track ids that will be added to the TB_df next
    being_added = need_adding['TB_id'].drop_duplicates().index.to_list()
    #we also need to remove these ids from the tracks that need_adding list
    updated_need_adding_ids = [track for track in need_adding.index if track not in being_added]
    need_adding = need_adding.loc[updated_need_adding_ids]
    
    #initialising df to be merged with COM_TB_df, also getting rid of pointless features such as those already in TB df
    feats = extra_COM_stuff + ['Eextra_track', 'TB_id']
    being_added_df = extra_tracks_COM_df.loc[being_added, feats]
    
    #need to change index of being_added_df so it can be merged with the TB_df
    being_added_df.index = being_added_df['TB_id']
    being_added_df = being_added_df.drop(columns=(['TB_id', 'TwoBody_Extra_TRUEPID','TwoBody_Extra_FromSameB']))
    #we also need to change names of the being_added_df columns as we will be adding more than one
    being_added_df.columns = [name +'_' + str(count) for name in being_added_df.columns ]
    #time to merge the being_added_df to the TB_df
    COM_TB_df = pd.concat([COM_TB_df, being_added_df], axis=1)
    
    count+= 1
      
    

2467
1391
736
385
204
112
64
29


In [21]:
COM_TB_df.SignalB_ID.unique()

array([-521,  521])

In [46]:
vector_features = ['pv_vector',
 'sv_vector',
 'flight',
'boost_est',
 'track1_p4',
 'track1_p4_boosted',
 'track2_p4',
 'track2_p4_boosted',
'p4B',
'p4B_est']

over_powered_features = ['TwoBody_OWNPV_X', 'TwoBody_OWNPV_Y', 'TwoBody_OWNPV_Z', 'TwoBody_ENDVERTEX_X', 'TwoBody_ENDVERTEX_Y', 'TwoBody_ENDVERTEX_Z']

In [79]:
feats =  [c for c in COM_TB_df.columns if c not in vector_features + labels] 
X = COM_TB_df[feats]
X = X.fillna(-1)
y = COM_TB_df['SignalB_ID']
print(y.unique())
#make y binary
y = y.replace(521, 1) ; y = y.replace(-521, 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=43)
ids1 = X_train.index ; ids2 = X_test.index

oof = pd.Series(np.zeros(y_train.shape[0]), index= ids1)
preds = pd.Series(np.zeros(y_test.shape[0]), index= ids2)

all_data = pd.concat([X_train, X_test])
norm_data = StandardScaler().fit_transform(all_data)
X_train = norm_data[:X_train.shape[0]] ; y_train = y_train.to_numpy().ravel()
X_test = norm_data[X_train.shape[0]:] ; y_test = y_test.to_numpy().ravel()

skf = StratifiedKFold(n_splits=5, random_state=42)


for train_idx, cv_idx in tqdm(skf.split(X_train, y_train), total=skf.n_splits):
    model = LGBMClassifier()
    model.fit(X_train[train_idx], y_train[train_idx])
    oof.iloc[cv_idx] = model.predict_proba(X_train[cv_idx])[:,1]
    preds.loc[ids2] += model.predict_proba(X_test)[:,1] / skf.n_splits

  0%|          | 0/5 [00:00<?, ?it/s]

[ 521 -521]


100%|██████████| 5/5 [00:03<00:00,  1.45it/s]


In [80]:
accuracy_score(y_train, round(oof))

0.7310924369747899

In [81]:
accuracy_score(y_test, round(preds))

0.7869318181818182

In [82]:
roc_auc_score(y_train, oof), precision_score(y_train, round(oof)) , recall_score(y_train, round(oof))

(0.8095757100112817, 0.7234600262123198, 0.7613793103448275)

In [83]:
roc_auc_score(y_test, preds), precision_score(y_test, round(preds)) , recall_score(y_test, round(preds))

(0.8683719238695599, 0.7977839335180056, 0.7890410958904109)

In [84]:
pd.Series(data=model.feature_importances_, index= X.columns).sort_values(ascending=False)[:50]

TwoBody_OWNPV_Y              176
TwoBody_OWNPV_X              173
TwoBody_OWNPV_Z              140
Track1_ProbNNk               112
Track2_ProbNNghost           101
Track1_ProbNNe                89
Track2_PT                     84
log(Track2_MINIPCHI2)         83
flight_phi                    82
Track2_ProbNNe                82
Track2_PY                     81
Track1_ProbNNghost            80
TwoBody_ENDVERTEX_Y           78
Track1_PZ                     78
TwoBody_FD_OWNPV              73
log(Track1_MINIPCHI2)         70
TwoBody_ENDVERTEX_Z           68
TwoBody_PT                    61
Track2_ProbNNk                58
Track2_ProbNNpi               58
Track1_PY                     58
TwoBody_ENDVERTEX_X           57
pt_est                        57
Etrack1                       55
Track2_PX                     54
Track1_ProbNNp                52
TwoBody_M                     52
Track2_ProbNNp                51
TwoBody_PX                    50
TwoBody_DIRA_OWNPV            49
Track2_PZ 

In [94]:
test = read_root('TaggingJpsiK2012_tiny_fix_fix.root', columns= ['TwoBody_FromSameB', 'SignalB_ID'] + ids )
#as eventNumber is not unique, but for a given run number, it is, so we can combine the two to get a completely unique id for each event
test.index = test.apply(lambda x:str(int(x.runNumber)) + str(int(x.eventNumber))+'-'+str(int(x.nCandidate)), axis=1 )
test = test.drop(columns=['runNumber', 'eventNumber', 'nCandidate'], axis=0)
test.SignalB_ID = test.SignalB_ID.replace(521, 1) ; test.SignalB_ID = test.SignalB_ID.replace(-521, 0)
test = test.loc[oof.index]
test['oof'] = round(oof)

In [95]:
test = test.query('TwoBody_FromSameB==1')

In [96]:
accuracy_score(test.SignalB_ID, test.oof)

0.8122653316645807

In [93]:
test

,TwoBody_FromSameB,SignalB_ID,oof
531373215304135-30,0.0,0,0.0
531373215304182-10,0.0,1,1.0
531404615849020-13,0.0,0,0.0
531373215304365-19,0.0,0,0.0
531404615848596-7,0.0,1,1.0
531373215304752-28,0.0,0,0.0
531404615848705-13,0.0,0,0.0
531404615849440-5,0.0,0,0.0
531404615849031-1,0.0,1,1.0
531373215304898-7,0.0,1,1.0
